Training is completed.<br>
Now test the model and display the results.<br>

In [ ]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models import ResNet18_Weights
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import logging, os, glob
from _logging import set_logging
from _metrics import display_metrics
from _pckle import save_pickle_object, load_pickle_object
from _utility import gl, get_perc, get_dictionaries_from_list
from _model import train_model

set_logging(logging)
logging.info(f"Cuda is available: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classes = ["Business", "Other"]
dict_classes, dict_classes_rev = get_dictionaries_from_list(classes)
test_loader = load_pickle_object(gl.pkl_test_loader)

In [ ]:
def imshow(inp, _mean, _std, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array(_mean)
    std = np.array(_std)
    inp = std * inp + mean  # denormalise
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
def visualize_model(model, test_loader, classes, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {classes[preds[j]]}')
                imshow(inputs.cpu().data[j])
                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
def get_best_model_from_checkpoint():
    checkpoint_path = os.path.join(gl.chk_dir, gl.chk_resnet_model)
    model_conv = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)
    checkpoint = torch.load(checkpoint_path)
    model_conv.load_state_dict(checkpoint["model_state_dict"])
    epoch = checkpoint["epoch"]
    accuracy = checkpoint["accuracy"]
    logging.info(f"retreiving the best model from the {epoch} epoch and an accuracy of {accuracy}")
    return model_conv


In [ ]:
model_conv = get_best_model_from_checkpoint()
visualize_model(model_conv, test_loader, classes)

plt.ioff()
plt.show()